In [46]:
import sys
sys.path.append('../src')
from gc_sampler import GetFeatures
import numpy as np
import pandas as pd

In [2]:
symbol ='TX_G0.TWF'
colo = 'twf'
dates = []
tick_size = 1.
gf = GetFeatures(symbol=symbol, colo=colo, dates=dates, tick_size=tick_size)
gf.readRawData(path = '../data')

Start reading market data...
Reading data finished!
Reading data time usage: 4.44867110252 s


In [3]:
df = gf.getTrainDataSet()
print df.shape

Generating training sets...

***** Processing data for 20171106 *****
Extracting other people's behaviors...
Extracting finished!
Extracting time: 6.30469703674 s
Extracting image features...
Extracting finished!
Extracting time: 5.27286505699 s
(163612, 6, 10, 4)
Extracting Rnn features...
Extracting finished!
Extracting time: 7.95537805557 s
(163612, 6)
(163612, 6)

***** Processing data for 20171113 *****
Extracting other people's behaviors...
Extracting finished!
Extracting time: 5.84087181091 s
Extracting image features...
Extracting finished!
Extracting time: 5.03017783165 s
(144130, 6, 10, 4)
Extracting Rnn features...
Extracting finished!
Extracting time: 6.98148179054 s
(144130, 6)
(144130, 6)

***** Processing data for 20171129 *****
Extracting other people's behaviors...
Extracting finished!
Extracting time: 4.48783111572 s
Extracting image features...
Extracting finished!
Extracting time: 4.82993102074 s
(144301, 6, 10, 4)
Extracting Rnn features...
Extracting finished!
Ext

(3267048, 6)
(3267048, 6)
(3267048, 6, 10, 4)
(3267048,)
Job finished!
Time usage: 620.99268508 s
(3267048, 4)


In [60]:
## Keras
from keras.layers import Input, Dropout, Dense, BatchNormalization, Activation, concatenate, GRU, Embedding, Flatten
from keras.layers import Flatten, Conv2D, MaxPooling2D
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping#, TensorBoard
from keras import backend as K
from keras import optimizers
from keras.optimizers import SGD
from keras import initializers
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from sklearn.cross_validation import train_test_split
from keras.utils import np_utils

In [61]:
dtrain, dvalid = train_test_split(df, random_state=17, train_size=0.7)
print dtrain.columns
print dtrain.img.values[0].shape

Index([u'askp', u'behav', u'bidp', u'img'], dtype='object')
(6, 10, 4)


In [62]:
# Function to compute class weights
def comp_cls_wts(y, pwr = 0.5):
    '''
    Used to compute class weights
    '''
    dic = {}
    for x in set(y):
        dic[x] = len(y)**pwr/list(y).count(x)**pwr
    return dic
cls_wts = comp_cls_wts(dtrain.behav.values, pwr=0.5)
print cls_wts

{0: 2.9623885112563246, 1: 2.897094042735395, 2: 8.143745622000402, 3: 8.140677186471262, 4: 2.7812450024034754, 5: 2.776525179715494, 6: 3.8168574598622405, 7: 3.768377882600665, 8: 1.7183196399596146}


In [65]:
def get_inout_data(dataset):
    X = {
        'ask': np.array(dataset.askp.tolist()).astype(np.int32),
        'bid': np.array(dataset.bidp.tolist()).astype(np.int32),
        'img': np.array(dataset.img.tolist()).astype(np.float32),
        'y': np_utils.to_categorical(dataset.behav.values, num_classes=9)
    }
    return X
%time tr = get_inout_data(dtrain)
%time cv = get_inout_data(dvalid)

CPU times: user 10.6 s, sys: 7.89 s, total: 18.5 s
Wall time: 18.5 s
CPU times: user 4.31 s, sys: 826 ms, total: 5.13 s
Wall time: 5.14 s


In [66]:
print tr['ask'].shape, tr['bid'].shape, tr['img'].shape, tr['y'].shape
MAX_PRICE = np.max([np.max(vec) for vec in tr['ask']] + [np.max(vec) for vec in cv['ask']]) + 5
print MAX_PRICE

(2286933, 6) (2286933, 6) (2286933, 6, 10, 4) (2286933, 9)
10882


In [67]:
ask = Input(shape=[tr['ask'][0].shape[0]], name = 'ask')
bid = Input(shape=[tr['bid'][0].shape[0]], name = 'bid')
print tr['ask'][0].shape[0]
img = Input(shape=(6, 10, 4), name = 'img')

6


In [68]:
emb_ask = Embedding(MAX_PRICE, 32)(ask)
emb_bid = Embedding(MAX_PRICE, 32)(bid)
rnn_ask = GRU(16)(emb_ask)
rnn_bid = GRU(16)(emb_bid)
fc_ask = Dropout(0.5) (Dense(64) (rnn_ask))
fc_bid = Dropout(0.5) (Dense(64) (rnn_bid))

In [69]:
cnn_img = Conv2D(32, kernel_size = (3, 3), padding = 'same', activation='relu') (img)
cnn_img = Dropout(0.25)(cnn_img)
cnn_img = Conv2D(64, kernel_size = (3, 3), padding = 'same', activation='relu') (cnn_img)
cnn_img = Dropout(0.25)(cnn_img)
fc_img = Flatten()(cnn_img)
fc_img = Dropout(0.5) (Dense(128) (fc_img))

In [70]:
main = concatenate([
    fc_ask,
    fc_bid,
    fc_img
])
main = Dropout(0.5) (Dense(128) (main))
main = Dropout(0.5) (Dense(32) (main))
output = Dense(9, activation="softmax") (main)

In [71]:
model = Model([ask, bid, img], output)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
img (InputLayer)                 (None, 6, 10, 4)      0                                            
____________________________________________________________________________________________________
conv2d_6 (Conv2D)                (None, 6, 10, 32)     1184                                         
____________________________________________________________________________________________________
dropout_14 (Dropout)             (None, 6, 10, 32)     0                                            
____________________________________________________________________________________________________
ask (InputLayer)                 (None, 6)             0                                            
___________________________________________________________________________________________

In [ ]:
N_epoch = 72
learning_rate = 0.04
decay_rate = 1./1.20
optimizer = SGD(learning_rate)
loss = 'categorical_crossentropy'
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
### Train the model
print("TRAINING BEGINS!")
    
## Using adaptive decaying learning rate
def scheduler(epoch):
    global learning_rate
    global decay_rate
    if epoch%6 == 0:
        learning_rate *= decay_rate
        print("CURRENT LEARNING RATE = ", learning_rate)
    return learning_rate
change_lr = LearningRateScheduler(scheduler)

TRAINING BEGINS!


In [ ]:
res = model.fit([tr['ask'], tr['bid'], tr['img']], tr['y'], batch_size = 128, epochs = N_epoch, 
                verbose = 1, validation_data = ([cv['ask'], cv['bid'], cv['img']], cv['y']), 
                class_weight = cls_wts, callbacks = [change_lr])

Train on 2286933 samples, validate on 980115 samples
('CURRENT LEARNING RATE = ', 0.03333333333333333)
Epoch 1/72
